In [ ]:
!pip install scikit-learn pandas joblib


Another execution

In [ ]:
##executing legal-BERT embeddings for logistic regression
##feature-1(legal clause classification)

!pip install transformers
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

##load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_extracted_clauses.csv")
clauses = df['Clause'].tolist()

In [ ]:
print(clauses)

['mutual confidentiality non disclosure agreement confidentiality nondisclosure agreement agreement date april effective date enter company incorporate provision company act have principal place business hereinafter refer', 'expression shall mean include parent affiliate sister concern subsidiary assign company incorporate provision company have principal place business', 'hereinafter refer company expression shall mean include parent affiliate sister concern subsidiary assign purpose party disclose certain confidential technical business information order avail gsp service company', 'protect say confidential information party desire sign non', 'disclosure confidential information party disclose party orally record medium information comprise relate affiliate parent sister concern group company technique schematic design contract financial information sale marketing plan business plan client client datum business affair operation strategy invention methodology technologie employee subc

In [ ]:
# Remove classes with less than 2 examples
class_counts = df['Clause Type'].value_counts()
df_filtered = df[df['Clause Type'].isin(class_counts[class_counts >= 2].index)]


In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_extracted_clauses.csv")
clauses = df_filtered['Clause'].tolist()
labels = df_filtered['Clause Type'].tolist()


In [ ]:
##rectified code
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch


class_counts = df['Clause Type'].value_counts()
df_filtered = df[df['Clause Type'].isin(class_counts[class_counts >= 2].index)].copy()

###clauses and labels from the filtered DataFrame
clauses = df_filtered['Clause'].tolist()
labels = df_filtered['Clause Type'].tolist()

##Ensure get_bert_embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()


##embeddings for the filtered data
embeddings = np.array([get_bert_embedding(text) for text in clauses])

##Fit LabelEncoder with labels from the filtered DataFrame
le = LabelEncoder()
y = le.fit_transform(labels)
print("Classes:", le.classes_)

##Split data
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, stratify=y, random_state=42)


clf = LogisticRegression(max_iter=1000, class_weight='balanced')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
# Use the fitted le.classes_ which now matches the classes in y_test
print(classification_report(y_test, y_pred, target_names=le.classes_[np.unique(y_test)], labels=np.unique(y_test)))

Classes: ['Breach and Remedies Clause' 'Communication/Notice Clause'
 'Confidentiality Clause' 'Confidentiality Obligation Clause'
 'Counterparts Clause' 'Definition Clause' 'Definitions Clause'
 'Disclosure to Representatives Clause' 'Exclusion Clause'
 'Exclusions from Confidentiality Clause' 'Execution Clause'
 'Governing Law Clause' 'Governing Law and Jurisdiction Clause'
 'Integration or Entire Agreement Clause' 'Introductory Clause'
 'Jurisdiction Clause' 'No License/No Implied Rights Clause'
 'No Obligation Clause' 'Non-Disclosure Clause' 'Notice Clause'
 'Obligations Clause' 'Other Clauses' 'Ownership Clause' 'Purpose Clause'
 'Remedies Clause' 'Reporting Clause'
 'Return or Destruction of Information Clause'
 'Secrecy of Agreement Clause' 'Security Clause' 'Severability Clause'
 'Successors and Assigns Clause' 'Term and Termination Clause'
 'Warranty Clause']
Accuracy: 0.34782608695652173
                                             precision    recall  f1-score   support

   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
df_filtered["Clause Type"].value_counts()

,count
Clause Type,
Obligations Clause,85
Definition Clause,46
Confidentiality Clause,26
Communication/Notice Clause,26
Counterparts Clause,25
Return or Destruction of Information Clause,24
Governing Law and Jurisdiction Clause,23
Integration or Entire Agreement Clause,22
Breach and Remedies Clause,22


In [ ]:
### adding feature for risk scoring

In [ ]:
import pandas as pd

###risk rubric dataset
rubric_data = [
    # Jurisdiction
    ("Jurisdiction", "No jurisdiction clause", "Omission of jurisdiction makes enforcement unclear", "High"),
    ("Jurisdiction", "Foreign jurisdiction mentioned", "Indian companies bound to foreign courts", "High"),
    ("Jurisdiction", "Indian jurisdiction mentioned", "Indian courts/arbitration specified", "Low"),

    # Governing Law
    ("Governing Law", "No governing law clause", "Ambiguity in applicable law", "High"),
    ("Governing Law", "Mentions foreign law", "Indian contract governed by foreign law", "High"),
    ("Governing Law", "Mentions Indian Contract Act", "Clearly within Indian legal scope", "Low"),

    # Termination
    ("Termination", "Only one party can terminate", "Unilateral termination rights", "High"),
    ("Termination", "No notice period", "No reasonable termination time", "High"),
    ("Termination", "Mutual termination rights + notice period", "Balanced and fair termination clause", "Low"),

    # Confidentiality
    ("Confidentiality", "No duration defined", "Endless confidentiality obligation", "Medium"),
    ("Confidentiality", "No exceptions listed", "Restrictive clause with no legal disclosures allowed", "Medium"),
    ("Confidentiality", "Time-bound + defined scope", "Clause is limited and clear", "Low"),

    # Indemnity
    ("Indemnity", "Unilateral indemnity", "Only one party bears all legal risk", "High"),
    ("Indemnity", "Unlimited liability", "No limit to indemnification", "High"),
    ("Indemnity", "Mutual indemnity + capped amount", "Balanced and reasonable", "Low"),

    # Force Majeure
    ("Force Majeure", "No force majeure clause", "No protection from natural/legal disruptions", "Medium"),
    ("Force Majeure", "Includes vague/avoidable events", "E.g., poor performance included", "Medium"),
    ("Force Majeure", "Standard events + Indian law wording", "Properly protects parties", "Low"),

    # Payment Terms
    ("Payment Terms", "No penalty for delay", "One party can delay without consequence", "Medium"),
    ("Payment Terms", "Ambiguous due dates", "Unclear cash flow obligations", "Medium"),
    ("Payment Terms", "Defined dates + interest penalty", "Fair and enforceable", "Low"),

    # Limitation of Liability
    ("Limitation of Liability", "No liability cap", "Unlimited liability exposure", "High"),
    ("Limitation of Liability", "No clause at all", "Entire liability remains uncapped", "High"),
    ("Limitation of Liability", "Reasonable cap + exclusions for fraud", "Standard protection", "Low"),

    # IP Rights
    ("IP Rights", "IP ownership not defined", "Can lead to disputes", "High"),
    ("IP Rights", "Perpetual license granted to one party", "One-sided IP ownership", "Medium"),
    ("IP Rights", "Clearly defined IP assignment", "Properly protects creators", "Low"),

    # Non-Compete
    ("Non-Compete", "Over 3 years or broad geography", "Legally unenforceable in India", "High"),
    ("Non-Compete", "Includes reasonable restrictions", "Within accepted norms", "Medium"),
    ("Non-Compete", "Omitted or includes <2 years in India", "Not risky or standard scope", "Low"),

    # Arbitration
    ("Arbitration", "No arbitration clause", "Disputes may lead to court battles", "High"),
    ("Arbitration", "Only one party controls arbitrator", "Unfair dispute resolution", "High"),
    ("Arbitration", "Bilateral, with seat in India", "Standard and fair", "Low"),

    # Notice Clause
    ("Notice Clause", "No communication method", "Leads to missed legal notices", "Medium"),
    ("Notice Clause", "Only postal address, no email", "Inflexible communication", "Medium"),
    ("Notice Clause", "Email + postal + timeline", "Clear and modern", "Low"),

    # Assignment
    ("Assignment", "Allows transfer without consent", "One-sided power shift", "Medium"),
    ("Assignment", "Prohibits assignment fully", "Can restrict commercial rights", "Medium"),
    ("Assignment", "Allows with consent", "Balanced", "Low"),
]


rubric_df = pd.DataFrame(rubric_data, columns=["clause_type", "clause_variant", "risk_reason", "risk_level"])
rubric_df.head(10)


,clause_type,clause_variant,risk_reason,risk_level
0,Jurisdiction,No jurisdiction clause,Omission of jurisdiction makes enforcement unc...,High
1,Jurisdiction,Foreign jurisdiction mentioned,Indian companies bound to foreign courts,High
2,Jurisdiction,Indian jurisdiction mentioned,Indian courts/arbitration specified,Low
3,Governing Law,No governing law clause,Ambiguity in applicable law,High
4,Governing Law,Mentions foreign law,Indian contract governed by foreign law,High
5,Governing Law,Mentions Indian Contract Act,Clearly within Indian legal scope,Low
6,Termination,Only one party can terminate,Unilateral termination rights,High
7,Termination,No notice period,No reasonable termination time,High
8,Termination,Mutual termination rights + notice period,Balanced and fair termination clause,Low
9,Confidentiality,No duration defined,Endless confidentiality obligation,Medium


In [ ]:
##risk level prediction model
##feature-2

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import joblib


In [ ]:
##Step 2: Load and Preprocess Dataset
df = pd.read_csv('risk_label_clauses.csv')
df = df.dropna(subset=['Clause', 'Risk Score'])



In [ ]:
##Filter out invalid rows
df = df[df['Risk Score'].isin(['Low', 'Medium', 'High'])]  # valid labels only


# Encode risk level labels
label_encoder = LabelEncoder()
df['risk_encoded'] = label_encoder.fit_transform(df['Risk Score'])  # e.g., High=0, Low=1, Medium=2

In [ ]:
###Step 3: TF-IDF Vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=3000, stop_words='english')
X = vectorizer.fit_transform(df['Clause'])
y = df['risk_encoded']


In [ ]:
##Step 4: Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [ ]:
##Step 5: Train Logistic Regression Model
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
###Step 6: Evaluate the Model
y_pred = model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📊 Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


✅ Accuracy: 0.611764705882353
📊 Classification Report:
               precision    recall  f1-score   support

        High       0.70      0.47      0.56        15
         Low       0.76      0.62      0.68        42
      Medium       0.46      0.68      0.55        28

    accuracy                           0.61        85
   macro avg       0.64      0.59      0.60        85
weighted avg       0.65      0.61      0.62        85



In [ ]:
##Save model and vectorizer
joblib.dump(model, 'risk_model.pkl')
joblib.dump(vectorizer, 'risk_vectorizer.pkl')
joblib.dump(label_encoder, 'risk_label_encoder.pkl')


['risk_label_encoder.pkl']

In [ ]:
##Function to Predict Risk for a New Clause
def predict_risk_level(clause_text):
    model = joblib.load('risk_model.pkl')
    vectorizer = joblib.load('risk_vectorizer.pkl')
    label_encoder = joblib.load('risk_label_encoder.pkl')

    X_input = vectorizer.transform([clause_text])
    pred = model.predict(X_input)
    return label_encoder.inverse_transform(pred)[0]

# Example
print("📝 Risk Prediction:", predict_risk_level("Upon termination of this agreement, the vendor shall delete or return all confidential information and confirm in writing."))


📝 Risk Prediction: Medium
